In [1]:
import sys
import math

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import scipy.stats as stats

from multiprocessing import cpu_count
from tqdm import tqdm, tqdm_notebook

#%matplotlib inline

In [2]:
def cprintf(df):
    if not isinstance(df, pd.DataFrame):
        try:
            df = df.to_frame()
        except:
            raise ValueError('Object cannot be coerced to df.')
    
    print('-'*79)
    print('Data frame information')
    print('-'*79)
    print(df.tail(5))
    print('-'*50)
    print(df.info())
    print('-'*79)

In [3]:
def getDailyVol(close,span0=100):
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0>0]
    df0 = pd.Series(close.index[df0-1], index = close.index[close.shape[0]-df0.shape[0]:])

    try:
        df0 = close.loc[df0.index] / close.loc[df0.values].values - 1
    except Exception as e:
        print ('error: {e}\n please confirm duplicate indices')

    df0 = df0.ewm(span=span0).std().rename('dailyVol')
    return df0

In [4]:
def getTEvents(gRaw, h):
    tEvents, sPos, sNeg = [], 0, 0
    diff = np.log(gRaw).diff().dropna().abs()

    for i in tqdm(diff.index[1:]):
        try:
            pos, neg = float(sPos + diff.loc[i]), float(sNeg + diff.loc[i])
        except Exception as e:
            print(e)
            print(sPos + diff.loc[i], type(sPos + diff.loc[i]))
            print(sNeg + diff.loc[i], type(sNeg + diff.loc[i]))
            break
        
        sPos, sNeg = max(0.,pos), min(0.,neg)
        if sNeg <- h:
            sNeg = 0
            tEvents.append(i)
        
        if sPos > h:
            sPos = 0
            tEvents.append(i)

    return pd.DatetimeIndex(tEvents)

In [5]:
def addVerticalBarrier(tEvents, close, numDays=1):
    t1 = close.index.searchsorted(tEvents + pd.Timedelta(days = numDays))
    t1 = t1[t1 < close.shape[0]]
    t1 = (pd.Series(close.index[t1], index = tEvents[:t1.shape[0]]))
    return t1

In [6]:
def applyPtSlOnT1(close,events,ptSl,molecule):
    # apply stop loss/profit taking, if it takes place before t1 (end of event)
    events_=events.loc[molecule]
    out=events_[['t1']].copy(deep=True)
    
    if ptSl[0]>0: 
        pt=ptSl[0]*events_['trgt']
    else: 
        pt=pd.Series(index=events.index) # NaNs
        
    if ptSl[1]>0: 
        sl=-ptSl[1]*events_['trgt']
    else: 
        sl=pd.Series(index=events.index) # NaNs
        
    for loc,t1 in events_['t1'].fillna(close.index[-1]).iteritems():
        df0=close[loc:t1] # path prices
        df0=(df0/close[loc]-1)*events_.at[loc,'side'] # path returns
        out.loc[loc,'sl']=df0[df0<sl[loc]].index.min() # earliest stop loss
        out.loc[loc,'pt']=df0[df0>pt[loc]].index.min() # earliest profit taking
    return out

#def applyPtSlOnT1(close, events, ptSl, molecule):
#    # apply stop loss / profit taking, if it takes place before t1 (end of event)
#    events_ = events.loc[molecule]
#    out = events_[['t1']].copy(deep=True)
    
#    if ptSl[0] > 0 :
#        pt = ptSl[0] * events_['trgt']
#    else:
#        pt = pd.Series(index = events.index)
    
#    if ptSl[1] > 0:
#        sl = -ptSl[1] * events_['trgt']
#    else:
#        sl = pd.Series(index = events.index)
        
#    for loc, t1 in events_['t1'].fillna(close.index[-1]).iteritems():
#        df0 = close[loc:t1]
#        df0 = (df0 / close[loc] - 1) * events_.at[loc, 'side']
#        out.loc[loc, 'sl'] = df0[df0 < sl[loc]].index.min()
#        out.loc[loc, 'pt'] = df0[df0 > pt[loc]].index.min()
        
#    return out

In [7]:
def getEvents(close, tEvents, ptSl, trgt, minRet, numThreads, t1=False, side=None):
    #1) get target
    trgt=trgt.loc[tEvents]
    trgt=trgt[trgt>minRet] # minRet
    
    #2) get t1 (max holding period)
    if t1 is False:t1=pd.Series(pd.NaT, index=tEvents)
        
    #3) form events object, apply stop loss on t1
    if side is None:
        side_,ptSl_ = pd.Series(1.,index=trgt.index), [ptSl[0],ptSl[0]]
    else: 
        side_, ptSl_ = side.loc[trgt.index],ptSl[:2]
        
    events=(pd.concat({'t1':t1,'trgt':trgt,'side':side_}, axis=1).dropna(subset=['trgt']))
    
    df0=mpPandasObj(func=applyPtSlOnT1,pdObj=('molecule',events.index),
                    numThreads=numThreads,close=close,events=events,
                    ptSl=ptSl_)
    
    events['t1']=df0.dropna(how='all').min(axis=1) # pd.min ignores nan
    
    if side is None:
        events=events.drop('side',axis=1)
        
    return events


    

In [8]:
import multiprocessing as mp
import datetime as dt

#________________________________
def reportProgress(jobNum,numJobs,time0,task):
    # Report progress as asynch jobs are completed
    msg=[float(jobNum)/numJobs, (time.time()-time0)/60.]
    msg.append(msg[1]*(1/msg[0]-1))
    timeStamp=str(dt.datetime.fromtimestamp(time.time()))
    msg=timeStamp+' '+str(round(msg[0]*100,2))+'% '+task+' done after '+ \
        str(round(msg[1],2))+' minutes. Remaining '+str(round(msg[2],2))+' minutes.'
    if jobNum<numJobs:sys.stderr.write(msg+'\r')
    else:sys.stderr.write(msg+'\n')
    return
#________________________________
def processJobs(jobs,task=None,numThreads=24):
    # Run in parallel.
    # jobs must contain a 'func' callback, for expandCall
    if task is None:task=jobs[0]['func'].__name__
    pool=mp.Pool(processes=numThreads)
    outputs,out,time0=pool.imap_unordered(expandCall,jobs),[],time.time()
    # Process asyn output, report progress
    for i,out_ in enumerate(outputs,1):
        out.append(out_)
        reportProgress(i,len(jobs),time0,task)
    pool.close();pool.join() # this is needed to prevent memory leaks
    return out

def linParts(numAtoms,numThreads):
    # partition of atoms with a single loop
    parts=np.linspace(0,numAtoms,min(numThreads,numAtoms)+1)
    parts=np.ceil(parts).astype(int)
    return parts

def nestedParts(numAtoms,numThreads,upperTriang=False):
    # partition of atoms with an inner loop
    parts,numThreads_=[0],min(numThreads,numAtoms)
    for num in range(numThreads_):
        part=1+4*(parts[-1]**2+parts[-1]+numAtoms*(numAtoms+1.)/numThreads_)
        part=(-1+part**.5)/2.
        parts.append(part)
    parts=np.round(parts).astype(int)
    if upperTriang: # the first rows are heaviest
        parts=np.cumsum(np.diff(parts)[::-1])
        parts=np.append(np.array([0]),parts)
    return parts

def mpPandasObj(func, pdObj, numThreads=24, mpBatches=1, linMols=True, **kargs):
    if linMols:
        parts = linParts(len(pdObj[1]), numThreads * mpBatches)
    else:
        parts = nestedParts(len(pdObj[1]), numThreads * mpBatches)
        
    jobs = []
    for i in range(1, len(parts)):
        job = {pdObj[0] : pdObj[1][parts[i-1]:parts[i]], 'func' : func}
        job.update(kargs)
        jobs.append(job)
    
    if numThreads == 1:
        out = processJobs_(jobs)
    else:
        out = processJobs(jobs, numThreads = numThreads)
        
    if isinstance(out[0], pd.DataFrame):
        df0 = pd.DataFrame()
    elif isinstance(out[0], pd.Series):
        df0 = pd.Series()
    else:
        return out
    
    for i in out:
        df0 = df0.append(i)
    
    df0 = df0.sort_index()
    return df0

def processJobs_(jobs):
    # Run jobs sequentially, for debugging
    out=[]
    for job in jobs:
        out_=expandCall(job)
        out.append(out_)
    return out

def expandCall(kargs):
    # Expand the arguments of a callback function, kargs['func']
    func=kargs['func']
    del kargs['func']
    out=func(**kargs)
    return out

In [9]:
def getBinsOld(events,close):
    #1) prices aligned with events
    events_=events.dropna(subset=['t1'])
    px=events_.index.union(events_['t1'].values).drop_duplicates()
    px=close.reindex(px,method='bfill')
    #2) create out object
    out=pd.DataFrame(index=events_.index)
    out['ret']=px.loc[events_['t1'].values].values/px.loc[events_.index]-1
    out['bin']=np.sign(out['ret'])
    # where out index and t1 (vertical barrier) intersect label 0
    try:
        locs = out.query('index in @t1').index
        out.loc[locs, 'bin'] = 0
    except:
        pass
    return out

def getBins(events, close):
    '''
    Compute event's outcome (including side information, if provided).
    events is a DataFrame where:
    -events.index is event's starttime
    -events['t1'] is event's endtime
    -events['trgt'] is event's target
    -events['side'] (optional) implies the algo's position side
    Case 1: ('side' not in events): bin in (-1,1) <-label by price action
    Case 2: ('side' in events): bin in (0,1) <-label by pnl (meta-labeling)
    '''
    #1) prices aligned with events
    events_=events.dropna(subset=['t1'])
    px=events_.index.union(events_['t1'].values).drop_duplicates()
    px=close.reindex(px,method='bfill')
    #2) create out object
    out=pd.DataFrame(index=events_.index)
    out['ret']=px.loc[events_['t1'].values].values/px.loc[events_.index]-1
    if 'side' in events_:out['ret']*=events_['side'] # meta-labeling
    out['bin']=np.sign(out['ret'])
    if 'side' in events_:out.loc[out['ret']<=0,'bin']=0 # meta-labeling
    return out

In [10]:
def dropLabels(events, minPct=.05):
    # apply weights, drop labels with insufficient examples
    while True:
        df0=events['bin'].value_counts(normalize=True)
        if df0.min()>minPct or df0.shape[0]<3:break
        print('dropped label: ', df0.argmin(),df0.min())
        events=events[events['bin']!=df0.argmin()]
    return events

In [11]:
df = pd.read_csv('data/dollar_bars.csv', index_col='dates').drop_duplicates()
df.index = pd.to_datetime(df.index)

cprintf(df)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                        amount        price  dollar_volume     volume
dates                                                                
2018-07-23 15:25:31 -40.000000  7700.000000 -308000.000000 -40.000000
2018-07-23 15:29:11  31.444802  7700.000000  242124.976632  31.444802
2018-07-23 15:32:09   8.000000  7719.000000   61752.000000   8.000000
2018-07-23 15:36:58   0.200000  7733.000000    1546.600000   0.200000
2018-07-23 15:40:01   0.996000  7731.245984    7700.321000   0.996000
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4943 entries, 2018-07-08 19:41:28 to 2018-07-23 15:40:01
Data columns (total 4 columns):
amount           4943 non-null float64
price            4943 non-null float64
dollar_volume    4943 non-null float64
volume           

In [12]:
close = df.price.copy()
close = close[~close.index.duplicated(keep='first')]

dailyVol = getDailyVol(close, 24)
cprintf(dailyVol.to_frame())

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     dailyVol
dates                        
2018-07-23 15:25:31  0.004040
2018-07-23 15:29:11  0.004130
2018-07-23 15:32:09  0.004008
2018-07-23 15:36:58  0.003973
2018-07-23 15:40:01  0.003834
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4485 entries, 2018-07-09 19:58:11 to 2018-07-23 15:40:01
Data columns (total 1 columns):
dailyVol    4484 non-null float64
dtypes: float64(1)
memory usage: 70.1 KB
None
-------------------------------------------------------------------------------


In [13]:
f,ax = plt.subplots()
dailyVol.plot(ax=ax)
ax.axhline(dailyVol.mean(), ls='--', color='r')


In [14]:
tEvents = getTEvents(close, h = dailyVol.mean())
tEvents

100%|██████████| 4665/4665 [00:00<00:00, 14256.72it/s] | 1302/4665 [00:00<00:00, 13014.27it/s]


DatetimeIndex(['2018-07-08 21:50:16', '2018-07-08 23:04:36',
               '2018-07-08 23:07:47', '2018-07-08 23:08:30',
               '2018-07-08 23:43:27', '2018-07-09 00:11:32',
               '2018-07-09 00:42:10', '2018-07-09 01:14:56',
               '2018-07-09 03:35:13', '2018-07-09 06:12:35',
               ...
               '2018-07-23 11:16:28', '2018-07-23 11:40:02',
               '2018-07-23 11:52:34', '2018-07-23 12:07:18',
               '2018-07-23 12:49:57', '2018-07-23 13:18:08',
               '2018-07-23 13:59:23', '2018-07-23 14:38:40',
               '2018-07-23 15:08:55', '2018-07-23 15:36:58'],
              dtype='datetime64[ns]', length=944, freq=None)

In [15]:
t1 = addVerticalBarrier(tEvents, close)
t1.index

DatetimeIndex(['2018-07-08 21:50:16', '2018-07-08 23:04:36',
               '2018-07-08 23:07:47', '2018-07-08 23:08:30',
               '2018-07-08 23:43:27', '2018-07-09 00:11:32',
               '2018-07-09 00:42:10', '2018-07-09 01:14:56',
               '2018-07-09 03:35:13', '2018-07-09 06:12:35',
               ...
               '2018-07-22 11:52:04', '2018-07-22 13:22:50',
               '2018-07-22 13:54:52', '2018-07-22 14:23:26',
               '2018-07-22 14:43:24', '2018-07-22 14:48:15',
               '2018-07-22 15:02:15', '2018-07-22 15:18:49',
               '2018-07-22 15:25:56', '2018-07-22 15:34:06'],
              dtype='datetime64[ns]', length=841, freq=None)

In [16]:
ptSl = [1,1]
target = dailyVol
minRet = 0.01
cpus = cpu_count() - 1

events = getEvents(close, tEvents, ptSl, target, minRet, cpus, t1=t1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()
2018-08-25 05:45:13.881899 66.67% applyPtSlOnT1 done after 0.0 minutes. Remaining 0.0 minutes.2018-08-25 05:45:13.882002 100.0% applyPtSlOnT1 done after 0.0 minutes. Remaining 0.0 minutes.


In [17]:
cprintf(events)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                                     t1      trgt
2018-07-20 16:40:28 2018-07-20 16:43:09  0.012766
2018-07-20 16:41:19 2018-07-20 16:58:36  0.011954
2018-07-20 17:03:43 2018-07-20 18:01:39  0.010749
2018-07-20 17:03:48 2018-07-20 18:01:39  0.011086
2018-07-20 17:04:32 2018-07-20 17:54:30  0.010653
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41 entries, 2018-07-11 11:44:37 to 2018-07-20 17:04:32
Data columns (total 2 columns):
t1      41 non-null datetime64[ns]
trgt    41 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 984.0 bytes
None
-------------------------------------------------------------------------------


In [18]:
labels = getBinsOld(events, close)
labels.bin.value_counts()

 1.0    19
-1.0    16
 0.0     6
Name: bin, dtype: int64

In [19]:
clean_labels = dropLabels(labels)
clean_labels.bin.value_counts()

 1.0    19
-1.0    16
 0.0     6
Name: bin, dtype: int64

In [20]:
%matplotlib

l0 = labels[labels['bin'] > 0]
l1 = labels[labels['bin'] < 0]

c0 = close.loc[l0.index]
c1 = close.loc[l1.index]

f,ax = plt.subplots()
close.plot(ax=ax)
c0.plot(ax=ax, ls = '', marker = '^', markersize = 7, color = 'g')
c1.plot(ax=ax, ls = '', marker = 'v', markersize = 7, color = 'r')


Using matplotlib backend: TkAgg


In [21]:
fast_window = 3
slow_window = 7

close_df = (pd.DataFrame().assign(price=close).assign(fast=close.ewm(fast_window).mean()).assign(slow=close.ewm(slow_window).mean()))
cprintf(close_df)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                           price         fast         slow
dates                                                     
2018-07-23 15:25:31  7706.600000  7703.899255  7704.372340
2018-07-23 15:29:11  7700.000000  7702.924441  7703.825798
2018-07-23 15:32:09  7719.000000  7706.943331  7705.722573
2018-07-23 15:36:58  7733.000000  7713.457498  7709.132251
2018-07-23 15:40:01  7731.245984  7717.904620  7711.896468
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4667 entries, 2018-07-08 19:41:28 to 2018-07-23 15:40:01
Data columns (total 3 columns):
price    4667 non-null float64
fast     4667 non-null float64
slow     4667 non-null float64
dtypes: float64(3)
memory usage: 145.8 KB
None
-----------------------------------------------------------------------

In [22]:
def get_up_cross(df):
    crit1 = df.fast.shift(1) < df.slow.shift(1)
    crit2 = df.fast > df.slow
    return df.fast[(crit1) & (crit2)]

def get_down_cross(df):
    crit1 = df.fast.shift(1) > df.slow.shift(1)
    crit2 = df.fast < df.slow
    return df.fast[(crit1) & (crit2)]

up = get_up_cross(close_df)
down = get_down_cross(close_df)

f, ax = plt.subplots(figsize=(11,8))

close_df.plot(ax=ax, alpha=.5)
up.plot(ax=ax, ls = '', marker='^', markersize=7, alpha=.75, label='upcross', color='g')
down.plot(ax=ax, ls = '', marker='v', markersize=7, alpha=.75, label='downcross', color='r')

ax.legend()

In [23]:
side_up = pd.Series(1, index=up.index)
side_down = pd.Series(-1, index=down.index)
side = pd.concat([side_up, side_down]).sort_index()
cprintf(side)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     0
dates                 
2018-07-23 12:01:20  1
2018-07-23 12:22:04 -1
2018-07-23 12:49:57  1
2018-07-23 14:38:40 -1
2018-07-23 15:32:09  1
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 285 entries, 2018-07-08 20:36:54 to 2018-07-23 15:32:09
Data columns (total 1 columns):
0    285 non-null int64
dtypes: int64(1)
memory usage: 4.5 KB
None
-------------------------------------------------------------------------------


In [24]:
minRet = .000001
ptsl=[1,2]
ma_events = getEvents(close, tEvents, ptsl, target, minRet, cpus, t1=t1, side=side)
ma_events.side.value_counts()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
2018-08-25 05:45:32.376507 66.67% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes..2018-08-25 05:45:32.408000 100.0% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes.


 1.0    40
-1.0    33
Name: side, dtype: int64

In [26]:
ma_side = ma_events.dropna().side
ma_bins = getBins(ma_events,close).dropna()

Xx = pd.merge_asof(ma_bins, side.to_frame().rename(columns={0: 'side'}), left_index=True, right_index=True, direction='forward')
cprintf(Xx)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                          ret  bin  side
2018-07-23 06:45:11  0.002901  1.0     1
2018-07-23 07:06:05 -0.008918  0.0    -1
2018-07-23 07:16:46  0.003747  1.0     1
2018-07-23 12:49:57  0.002389  1.0     1
2018-07-23 14:38:40 -0.004481  0.0    -1
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 73 entries, 2018-07-09 21:28:34 to 2018-07-23 14:38:40
Data columns (total 3 columns):
ret     73 non-null float64
bin     73 non-null float64
side    73 non-null int64
dtypes: float64(2), int64(1)
memory usage: 2.3 KB
None
-------------------------------------------------------------------------------


In [27]:
%matplotlib

l0 = Xx[Xx['side'] > 0]
l1 = Xx[Xx['side'] < 0]

c0 = close.loc[l0.index]
c1 = close.loc[l1.index]

f,ax = plt.subplots()
close.plot(ax=ax)
c0.plot(ax=ax, ls = '', marker = '^', markersize = 7, color = 'g')
c1.plot(ax=ax, ls = '', marker = 'v', markersize = 7, color = 'r')

Using matplotlib backend: TkAgg


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report

In [49]:
RANDOM_STATE = 777

X = ma_side.values.reshape(-1,1)
y = ma_bins.bin.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
n_estimator = 100

rf = RandomForestClassifier(max_depth=3, n_estimators=n_estimator, criterion='entropy', random_state=RANDOM_STATE)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=777,
            verbose=0, warm_start=False)

In [52]:
y_pred_rf = rf.predict_proba(X_test)[:,1]
y_pred = rf.predict(X_test)

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred)
print(classification_report(y_test, y_pred))

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

             precision    recall  f1-score   support

        0.0       0.59      0.62      0.61        16
        1.0       0.70      0.67      0.68        21

avg / total       0.65      0.65      0.65        37



In [53]:
def bbands(price, window=None, width=None, numsd=None):
    """ returns average, upper band, and lower band"""
    ave = price.rolling(window).mean()
    sd = price.rolling(window).std(ddof=0)
    if width:
        upband = ave * (1+width)
        dnband = ave * (1-width)
        return price, np.round(ave,3), np.round(upband,3), np.round(dnband,3)        
    if numsd:
        upband = ave + (sd*numsd)
        dnband = ave - (sd*numsd)        
        return price, np.round(ave,3), np.round(upband,3), np.round(dnband,3)

In [55]:
window=50
bb_df = pd.DataFrame()
bb_df['price'],bb_df['ave'],bb_df['upper'],bb_df['lower']=bbands(close, window=window, numsd=1)
bb_df.dropna(inplace=True)
cprintf(bb_df)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                           price       ave     upper     lower
dates                                                         
2018-07-23 15:25:31  7706.600000  7693.472  7712.102  7674.842
2018-07-23 15:29:11  7700.000000  7693.970  7712.434  7675.506
2018-07-23 15:32:09  7719.000000  7694.952  7713.416  7676.489
2018-07-23 15:36:58  7733.000000  7696.432  7714.920  7677.944
2018-07-23 15:40:01  7731.245984  7697.819  7716.266  7679.372
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4618 entries, 2018-07-09 04:58:17 to 2018-07-23 15:40:01
Data columns (total 4 columns):
price    4618 non-null float64
ave      4618 non-null float64
upper    4618 non-null float64
lower    4618 non-null float64
dtypes: float64(4)
memory usage: 180.4 KB
None
------------

In [56]:
f,ax=plt.subplots(figsize=(11,8))
bb_df.plot(ax=ax)

In [57]:
def get_up_cross(df, col):
    # col is price column
    crit1 = df[col].shift(1) < df.upper.shift(1)  
    crit2 = df[col] > df.upper
    return df[col][(crit1) & (crit2)]

def get_down_cross(df, col):
    # col is price column    
    crit1 = df[col].shift(1) > df.lower.shift(1) 
    crit2 = df[col] < df.lower
    return df[col][(crit1) & (crit2)]

bb_down = get_down_cross(bb_df, 'price')
bb_up = get_up_cross(bb_df, 'price') 

f, ax = plt.subplots(figsize=(11,8))

bb_df.plot(ax=ax, alpha=.5)
bb_up.plot(ax=ax, ls='', marker='^', markersize=7, alpha=0.75, label='upcross', color='g')
bb_down.plot(ax=ax, ls='', marker='v', markersize=7,alpha=0.75, label='downcross', color='r')
ax.legend()

In [63]:
bb_side_up = pd.Series(-1, index=bb_up.index) # sell on up cross for mean reversion
bb_side_down = pd.Series(1, index=bb_down.index) # buy on down cross for mean reversion
bb_side_raw = pd.concat([bb_side_up,bb_side_down]).sort_index()
cprintf(bb_side_raw)

minRet = .001 
ptsl=[0,2]
bb_events = getEvents(close,tEvents,ptsl,target,minRet,cpus,t1=t1,side=bb_side_raw)
cprintf(bb_events)

bb_side = bb_events.dropna().side
cprintf(bb_side)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     0
dates                 
2018-07-23 11:37:10  1
2018-07-23 12:18:28  1
2018-07-23 12:31:42  1
2018-07-23 12:53:28 -1
2018-07-23 15:32:09 -1
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 356 entries, 2018-07-09 07:29:13 to 2018-07-23 15:32:09
Data columns (total 1 columns):
0    356 non-null int64
dtypes: int64(1)
memory usage: 5.6 KB
None
-------------------------------------------------------------------------------


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     side  t1      trgt
2018-07-23 13:18:08   NaN NaT  0.002966
2018-07-23 13:59:23   NaN NaT  0.002460
2018-07-23 14:38:40   NaN NaT  0.002215
2018-07-23 15:08:55   NaN NaT  0.003308
2018-07-23 15:36:58   NaN NaT  0.003973
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 914 entries, 2018-07-09 21:28:34 to 2018-07-23 15:36:58
Data columns (total 3 columns):
side    115 non-null float64
t1      819 non-null datetime64[ns]
trgt    914 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 28.6 KB
None
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Data frame information
-------------------------------------------

2018-08-25 07:37:18.100752 100.0% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes.


In [64]:
bb_side.value_counts()

-1.0    56
 1.0    53
Name: side, dtype: int64

In [68]:
bb_bins = getBins(bb_events,close).dropna()
cprintf(bb_bins)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                          ret  bin
2018-07-22 21:20:18 -0.008944  0.0
2018-07-22 22:49:24 -0.007765  0.0
2018-07-23 04:30:35 -0.008096  0.0
2018-07-23 06:48:50 -0.010121  0.0
2018-07-23 08:05:22 -0.004784  0.0
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 109 entries, 2018-07-09 23:15:16 to 2018-07-23 08:05:22
Data columns (total 2 columns):
ret    109 non-null float64
bin    109 non-null float64
dtypes: float64(2)
memory usage: 2.6 KB
None
-------------------------------------------------------------------------------


In [69]:
bb_bins.bin.value_counts()

0.0    93
1.0    16
Name: bin, dtype: int64

In [70]:
def returns(s):
    arr = np.diff(np.log(s))
    return (pd.Series(arr, index=s.index[1:]))

def df_rolling_autocorr(df, window, lag=1):
    """Compute rolling column-wise autocorrelation for a DataFrame."""

    return (df.rolling(window=window)
            .corr(df.shift(lag))) # could .dropna() here

#df_rolling_autocorr(d1, window=21).dropna().head()

In [72]:
srl_corr = df_rolling_autocorr(returns(close), window=window).rename('srl_corr')
cprintf(srl_corr)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     srl_corr
dates                        
2018-07-23 15:25:31 -0.009523
2018-07-23 15:29:11 -0.019654
2018-07-23 15:32:09 -0.073255
2018-07-23 15:36:58 -0.017369
2018-07-23 15:40:01 -0.024611
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4666 entries, 2018-07-08 20:16:45 to 2018-07-23 15:40:01
Data columns (total 1 columns):
srl_corr    4616 non-null float64
dtypes: float64(1)
memory usage: 72.9 KB
None
-------------------------------------------------------------------------------


In [74]:
features = (pd.DataFrame()
            .assign(vol=bb_events.trgt)
            .assign(ma_side=ma_side)
            .assign(srl_corr=srl_corr)
            .drop_duplicates()
            .dropna())
cprintf(features)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                          vol  ma_side  srl_corr
2018-07-23 06:45:11  0.002585      1.0 -0.016879
2018-07-23 07:06:05  0.004033     -1.0 -0.151176
2018-07-23 07:16:46  0.002935      1.0 -0.333736
2018-07-23 12:49:57  0.001945      1.0 -0.093937
2018-07-23 14:38:40  0.002215     -1.0  0.112511
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 73 entries, 2018-07-09 21:28:34 to 2018-07-23 14:38:40
Data columns (total 3 columns):
vol         73 non-null float64
ma_side     73 non-null float64
srl_corr    73 non-null float64
dtypes: float64(3)
memory usage: 2.3 KB
None
-------------------------------------------------------------------------------


In [76]:
Xy = (pd.merge_asof(features, bb_bins[['bin']], 
                    left_index=True, right_index=True, 
                    direction='forward').dropna())
cprintf(Xy)

-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                          vol  ma_side  srl_corr  bin
2018-07-23 03:37:33  0.002462     -1.0 -0.065742  0.0
2018-07-23 04:30:35  0.003797      1.0 -0.071414  0.0
2018-07-23 06:45:11  0.002585      1.0 -0.016879  0.0
2018-07-23 07:06:05  0.004033     -1.0 -0.151176  0.0
2018-07-23 07:16:46  0.002935      1.0 -0.333736  0.0
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71 entries, 2018-07-09 21:28:34 to 2018-07-23 07:16:46
Data columns (total 4 columns):
vol         71 non-null float64
ma_side     71 non-null float64
srl_corr    71 non-null float64
bin         71 non-null float64
dtypes: float64(4)
memory usage: 2.8 KB
None
-------------------------------------------------------------------------------


In [77]:
Xy.bin.value_counts()

0.0    60
1.0    11
Name: bin, dtype: int64

In [87]:
X = Xy.drop('bin',axis=1).values
y = Xy['bin'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

n_estimator = 1000
rf = RandomForestClassifier(max_depth=10, n_estimators=n_estimator,
                            criterion='entropy', random_state=RANDOM_STATE)
rf.fit(X_train, y_train)

# The random forest model by itself
y_pred_rf = rf.predict_proba(X_test)[:, 1]
y_pred = rf.predict(X_test)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)
print(classification_report(y_test, y_pred, target_names=['no_trade','trade']))

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

             precision    recall  f1-score   support

   no_trade       0.50      0.18      0.27        11
      trade       0.67      0.90      0.77        20

avg / total       0.61      0.65      0.59        31



In [88]:
minRet = .001 
ptsl=[0,2]
bb_events = getEvents(close,tEvents,ptsl,target,minRet,cpus,t1=t1)
cprintf(bb_events)

bb_bins = getBins(bb_events,close).dropna()
cprintf(bb_bins)

features = (pd.DataFrame()
            .assign(vol=bb_events.trgt)
            .assign(ma_side=ma_side)
            .assign(srl_corr=srl_corr)
            .drop_duplicates()
            .dropna())
cprintf(features)

Xy = (pd.merge_asof(features, bb_bins[['bin']], 
                    left_index=True, right_index=True, 
                    direction='forward').dropna())
cprintf(Xy)

### run model ###
X = Xy.drop('bin',axis=1).values
y = Xy['bin'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

n_estimator = 100
rf = RandomForestClassifier(max_depth=4, n_estimators=n_estimator,
                            criterion='entropy', random_state=RANDOM_STATE)
rf.fit(X_train, y_train)

# The random forest model by itself
y_pred_rf = rf.predict_proba(X_test)[:, 1]
y_pred = rf.predict(X_test)
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)
print(classification_report(y_test, y_pred))

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  app.launch_new_instance()
2018-08-26 01:13:08.503063 66.67% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes..2018-08-26 01:13:08.522102 100.0% applyPtSlOnT1 done after 0.01 minutes. Remaining 0.0 minutes.


-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                     t1      trgt
2018-07-23 13:18:08 NaT  0.002966
2018-07-23 13:59:23 NaT  0.002460
2018-07-23 14:38:40 NaT  0.002215
2018-07-23 15:08:55 NaT  0.003308
2018-07-23 15:36:58 NaT  0.003973
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 914 entries, 2018-07-09 21:28:34 to 2018-07-23 15:36:58
Data columns (total 2 columns):
t1      811 non-null datetime64[ns]
trgt    914 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 21.4 KB
None
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Data frame information
-------------------------------------------------------------------------------
                          re